In [ ]:
from pathlib import Path

In [ ]:
# Installation in case running over Colab
try:
    import google.colab
    on_colab = True
    %pip install estival
    %pip install pylatex==1.4.1
    %pip install kaleido
    ! git clone https://github.com/monash-emu/aust-covid
    %cd aust-covid
    %pip install -e ./
    PROJECT_PATH = Path().resolve()
    import multiprocessing as mp
    mp.set_start_method("forkserver")
except:
    PROJECT_PATH = Path().resolve().parent
    on_colab = False
optimise_model = False
new_calibration = False

DATA_PATH = PROJECT_PATH / "data"
OUTPUT_PATH = PROJECT_PATH / "outputs"
SUPPLEMENT_PATH = PROJECT_PATH / "supplement"
Path(OUTPUT_PATH).mkdir(parents=True, exist_ok=True)

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime, date
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import yaml
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm

from aust_covid import model
from doc_utils import calib_doc_utils
from aust_covid.inputs import load_param_info, load_calibration_targets
from doc_utils.general_doc_utils import TextElement, TableElement, FigElement, add_element_to_document, \
    save_pyplot_add_to_doc, save_plotly_add_to_doc, compile_doc, generate_doc

In [ ]:
from summer2.functions.time import get_linear_interpolation_function, Function
from summer2.parameters import Parameter

In [ ]:
smoothed_composite_cases, targets_text = load_calibration_targets(datetime(2021, 12, 15))
targets_text

In [ ]:
# Times
start_date = datetime(2021, 9, 1)  # Analysis start time
end_date = datetime(2022, 10, 1)  # Analysis end time
plot_start_date = datetime(2021, 12, 1)  # Left end for plots
ref_date = datetime(2019, 12, 31)  # Arbitrary reference date

In [ ]:
# Parameters
parameters = {
    'contact_rate': 0.0458, 
    'infectious_period': 6.898, 
    'start_cdr': 0.0997, 
    'ba1_seed_time': 657.4, 
    'natural_immunity_period': 23.5,
    "ba2_seed_time": 700.0,
    "ba2_escape": 0.8,
    "ba5_seed_time": 765., 
    "ba5_escape": 1.0,
    "latent_period": 2.0,
    "seed_rate": 1.0,
    "seed_duration": 1.0,
    "notifs_shape": 2.0,
    "notifs_mean": 4.0,
}
param_info = load_param_info(PROJECT_PATH / "inputs/parameters.yml", parameters)
param_info

In [ ]:
doc_sections = {}
compartments = [
    "susceptible",
    "latent",
    "infectious",
    "recovered",
    "waned",
]
aust_model, build_text = model.build_base_model(ref_date, compartments, start_date, end_date)
add_element_to_document("Model construction", TextElement(build_text), doc_sections)
build_text

In [ ]:
pop_data, pop_text = model.get_pop_data()
add_element_to_document("Model construction", TextElement(pop_text), doc_sections, subsection_name="Population")
pop_text

In [ ]:
start_text = model.set_starting_conditions(aust_model, pop_data, adjuster=1.0)
add_element_to_document("Model construction", TextElement(start_text), doc_sections, subsection_name="Population")
start_text

In [ ]:
infect_text = model.add_infection(aust_model)
add_element_to_document("Model construction", TextElement(infect_text), doc_sections)
infect_text

In [ ]:
prog_text = model.add_progression(aust_model)
add_element_to_document("Model construction", TextElement(prog_text), doc_sections)
prog_text

In [ ]:
rec_text = model.add_recovery(aust_model)
add_element_to_document("Model construction", TextElement(rec_text), doc_sections)
rec_text

In [ ]:
wane_text = model.add_waning(aust_model)
add_element_to_document("Model construction", TextElement(wane_text), doc_sections)
wane_text

In [ ]:
age_strata = list(range(0, 80, 5))
# raw_matrix, age_text, raw_matrix_fig, raw_matrix_name, raw_matrix_caption = model.build_polymod_britain_matrix(age_strata)
# add_element_to_document("Age stratification", TextElement(age_text), doc_sections, subsection_name="Mixing")
# add_element_to_document("Age stratification", FigElement(raw_matrix_name, caption=raw_matrix_caption), doc_sections, subsection_name="Mixing")
# age_text

In [ ]:
raw_mob_df, raw_mob_fig, raw_mob_filename, raw_mob_text = model.get_raw_mobility(plot_start_date, aust_model)
add_element_to_document("Mobility", TextElement(raw_mob_text), doc_sections)
raw_mob_text

In [ ]:
add_element_to_document("Mobility", FigElement(raw_mob_filename), doc_sections)
raw_mob_fig

In [ ]:
mean_smoothed_non_resi_mob, modelled_mob_fig, modelled_mob_filename = model.process_mobility(raw_mob_df, plot_start_date, aust_model)
add_element_to_document("Mobility", FigElement(modelled_mob_filename), doc_sections)
modelled_mob_fig

In [ ]:
mobility_adjuster, infection_func, mob_adj_text, mob_effect_fig, mob_effect_filename = model.calculate_mobility_effect(mean_smoothed_non_resi_mob, plot_start_date, aust_model)
add_element_to_document("Mobility", TextElement(mob_adj_text), doc_sections)
mob_adj_text

In [ ]:
add_element_to_document("Mobility", FigElement(mob_effect_filename), doc_sections)
mob_effect_fig

In [ ]:
mobility_scaling, mob_map_text = model.get_mobility_mapper()
add_element_to_document("Mobility", TextElement(mob_map_text), doc_sections)
mob_map_text

In [ ]:
locations = ["school", "home", "work", "other_locations"]
raw_location_matrices = {i: pd.read_csv(DATA_PATH / f"{i}.csv", index_col=0).to_numpy() for i in locations}
input_pop_fig, input_pop_caption, input_pop_filename, modelled_pop_fig, modelled_pop_caption, modelled_pop_filename, \
    matrix_ref_pop_fig, matrix_ref_pop_caption, matrix_ref_pop_filename, \
    adjusted_matrices, pop_splits = model.adapt_gb_matrices_to_aust(age_strata, raw_location_matrices, pop_data)
mixing_matrix = Function(
    mobility_scaling, 
    [
        adjusted_matrices,
        infection_func,
    ]
)

In [ ]:
input_pop_fig.layout.title.update({"text": input_pop_caption})
input_pop_fig

In [ ]:
modelled_pop_fig.layout.title.update({"text": modelled_pop_caption})
modelled_pop_fig

In [ ]:
matrix_ref_pop_fig.layout.title.update({"text": matrix_ref_pop_caption})
matrix_ref_pop_fig

In [ ]:
import numpy as np


In [ ]:
# from plotly.subplots import make_subplots
# import plotly.graph_objects as go

# def plot_mixing_matrices(matrices):
#     matrix_figsize = 800
#     fig = make_subplots(rows=2, cols=2, subplot_titles=locations)
#     for i_loc, loc in enumerate(locations):
#         fig.add_trace(
#             go.Heatmap(x=age_strata, y=age_strata, z=matrices[loc], coloraxis = "coloraxis"), 
#             int(np.floor(i_loc / 2)) + 1, 
#             i_loc % 2 + 1,
#         )
#     fig.update_layout(width=matrix_figsize, height=matrix_figsize * 1.15)
#     return fig

In [ ]:
model.plot_mixing_matrices(adjusted_matrices, locations, age_strata)

In [ ]:
# input_pop_filename = "input_population.jpg"
# input_pop_fig = px.bar(aust_pop_series, labels={"value": "population", "Age (years)": ""})
# input_pop_fig.update_layout(showlegend=False)
# input_pop_fig.write_image(SUPPLEMENT_PATH / input_pop_filename)
# input_pop_caption = "Australian population sizes by age group obtained from Australia Bureau of Statistics."

In [ ]:
# adjusted_matrix, pop_splits, mat_adj_text, \
#     input_pop_filename, input_pop_caption, input_pop_fig, modelled_pop_filename, modelled_pop_caption, modelled_pop_fig, \
#     matrix_ref_pop_filename, matrix_ref_pop_caption, matrix_ref_pop_fig, \
#     adjusted_matrix_filename, adjusted_matrix_caption, adjusted_matrix_fig, modelled_pop = model.adapt_gb_matrix_to_aust(age_strata, raw_matrix, pop_data)
# add_element_to_document("Age stratification", TextElement(mat_adj_text), doc_sections, subsection_name="Mixing")
# add_element_to_document("Age stratification", FigElement(input_pop_filename, caption=input_pop_caption), doc_sections, subsection_name="Mixing")
# add_element_to_document("Age stratification", FigElement(modelled_pop_filename, caption=modelled_pop_caption), doc_sections, subsection_name="Mixing")
# add_element_to_document("Age stratification", FigElement(matrix_ref_pop_filename, caption=matrix_ref_pop_caption), doc_sections, subsection_name="Mixing")
# add_element_to_document("Age stratification", FigElement(adjusted_matrix_filename, caption=adjusted_matrix_caption), doc_sections, subsection_name="Mixing")
# mat_adj_text

In [ ]:
age_strat, agestrat_text = model.add_age_stratification(compartments, age_strata, pop_splits, mixing_matrix)
aust_model.stratify_with(age_strat)
add_element_to_document("Age stratification", TextElement(agestrat_text), doc_sections)
agestrat_text

In [ ]:
strain_strata = ["ba1", "ba2", "ba5"]
strain_strat, strainstrat_text = model.get_strain_stratification(compartments, strain_strata)
aust_model.stratify_with(strain_strat)
add_element_to_document("Strain stratification", TextElement(strainstrat_text), doc_sections)
strainstrat_text

In [ ]:
seed_text = model.seed_vocs(aust_model)
add_element_to_document("Strain stratification", TextElement(seed_text), doc_sections)
seed_text

In [ ]:
reinfect_text = model.add_reinfection(aust_model, strain_strata, infection_func)
add_element_to_document("Strain stratification", TextElement(reinfect_text), doc_sections)
reinfect_text

In [ ]:
infection_processes = [
    "infection", 
    "early_reinfection",
    "late_reinfection",
]
inc_text = model.add_incidence_output(aust_model, infection_processes)
add_element_to_document("Outputs", TextElement(inc_text), doc_sections)
inc_text

In [ ]:
ratio_df, survey_fig, survey_fig_name, survey_fig_caption, ratio_fig, ratio_fig_name, ratio_fig_caption, cdr_description = model.add_notifications_output(aust_model)
add_element_to_document("Outputs", TextElement(cdr_description), doc_sections, subsection_name="Notifications")
add_element_to_document("Outputs", FigElement(survey_fig_name, caption=survey_fig_caption), doc_sections, subsection_name="Notifications")
add_element_to_document("Outputs", FigElement(ratio_fig_name, caption=ratio_fig_caption), doc_sections, subsection_name="Notifications")
cdr_description

In [ ]:
survey_fig.layout.title.update({"text": survey_fig_caption})
survey_fig

In [ ]:
ratio_fig.layout.title.update({"text": ratio_fig_caption})
ratio_fig

In [ ]:
model.track_sero_prevalence(compartments, aust_model)

In [ ]:
strain_prop_text = model.track_strain_prop(strain_strata, aust_model)
add_element_to_document("Outputs", TextElement(strain_prop_text), doc_sections, subsection_name="Other outputs")
strain_prop_text

In [ ]:
# Calibration/optimisation settings
priors = [
    UniformPrior("contact_rate", (0.035, 0.06)),
    UniformPrior("infectious_period", (4.0, 10.0)),
    UniformPrior("start_cdr", (0.05, 0.3)),
    UniformPrior("ba2_escape", (0.3, 1.0)),
    UniformPrior("ba5_escape", (0.3, 1.0)),
    UniformPrior("ba1_seed_time", (640.0, 670.0)),
    UniformPrior("ba2_seed_time", (680.0, 720.0)),
    UniformPrior("ba5_seed_time", (750.0, 800.0)),
    UniformPrior("natural_immunity_period", (20.0, 100.0)),
]
smoothed_composite_cases_intindex = smoothed_composite_cases.copy()
smoothed_composite_cases_intindex.index=(smoothed_composite_cases.index - aust_model.ref_date).days
def least_squares(modelled, obs, parameters, time_weights):
    return 0.0 - (((modelled - obs) ** 2.0)).sum()
targets = [
    CustomTarget("notifications", smoothed_composite_cases_intindex, least_squares),
]
binom_targets = [
    NegativeBinomialTarget("notifications", smoothed_composite_cases_intindex, 500.0),
]
calibration_model = BayesianCompartmentalModel(aust_model, parameters, priors, binom_targets)

In [ ]:
if optimise_model:
    print("Optimising with nevergrad \n Progression of loss function values:")
    optim_runner = optimize_model(calibration_model)
    for i in range(10):
        rec = optim_runner.minimize(100)
        print(rec.loss)
    optim_params = rec.value[1]
    parameters.update(optim_params)
    aust_model.run(parameters=parameters)
    print("Best calibration parameters found:")
    print(optim_params)

# Single run point

In [ ]:
aust_model.run(parameters=parameters)
axis_labels = {"index": "time", "value": "cases"}
comparison_df = pd.concat((smoothed_composite_cases, aust_model.get_derived_outputs_df()["notifications"]), axis=1)
comparison_df.plot(labels=axis_labels, title="Optimised parameter outputs")

In [ ]:
assert False

In [ ]:
strain_prop_fig, strain_prop_fig_name, strain_prop_fig_caption = model.show_strain_props(strain_strata, date(2021, 11, 1), aust_model)
add_element_to_document("Outputs", FigElement(strain_prop_fig_name, caption=strain_prop_fig_caption), doc_sections, subsection_name="Sub-variants")
strain_prop_fig.layout.title.update({"text": strain_prop_fig_caption})
strain_prop_fig

In [ ]:
# Main calibration loop
iterations = 10000
burn_in = 2000
n_chains = 10
if new_calibration:
    with pm.Model() as pm_model:
        variables = epm.use_model(calibration_model)
        idata_raw = pm.sample(step=[pm.DEMetropolis(variables)], draws=iterations, tune=0, cores=8, chains=n_chains)
    idata_raw.to_netcdf(OUTPUT_PATH / "calibration_out.nc")
else:
    idata_raw = az.from_netcdf(OUTPUT_PATH / "calibration_out.nc")

idata = idata_raw.sel(draw=range(burn_in, iterations))  # Discard burn-in

In [ ]:
# Check parameter starting points by chain
idata_raw.posterior.isel(draw=0).to_dataframe()

In [ ]:
# Report acceptance ratios by chain
(idata.sample_stats.accepted.sum(axis=1) / idata.sample_stats.coords["draw"].size).to_dataframe()

In [ ]:
priors_table = calib_doc_utils.tabulate_priors(priors, param_info)
add_element_to_document("Calibration", TableElement(priors_table), doc_sections)
priors_table

In [ ]:
calib_table = calib_doc_utils.tabulate_param_results(idata, priors, param_info)
add_element_to_document("Calibration", TableElement(calib_table), doc_sections)
calib_table

In [ ]:
param_table = calib_doc_utils.tabulate_parameters(parameters, priors, param_info)
add_element_to_document("Parameters", TableElement(param_table, col_requests=[0.25, 0.25, 0.5]), doc_sections)

In [ ]:
chains_plot = calib_doc_utils.plot_param_progression(idata, param_info)
save_pyplot_add_to_doc(chains_plot, "chains", "Calibration", doc_sections, caption="Parameter progression and posterior by chain.")

In [ ]:
posterior_plot = calib_doc_utils.plot_param_posterior(idata, param_info)
save_pyplot_add_to_doc(posterior_plot, "posterior", "Calibration", doc_sections, caption="Final estimated parameter posteriors.")

In [ ]:
sample_plot, sampled_df = calib_doc_utils.plot_sampled_outputs(idata, 30, "notifications", calibration_model, smoothed_composite_cases, plot_start_date, end_date)
save_plotly_add_to_doc(sample_plot, "calibration_fit", "Calibration", doc_sections, caption="Sampled model run fits to calibration targets")
sample_plot

In [ ]:
modelled_cdr_fig, modelled_cdr_fig_name, modelled_cdr_fig_caption = model.show_cdr_profiles(sampled_df["start_cdr"], ratio_df)
save_plotly_add_to_doc(modelled_cdr_fig, modelled_cdr_fig_name, "Calibration", doc_sections, caption=modelled_cdr_fig_caption)
modelled_cdr_fig

In [ ]:
# Complete the documentation process
supplement = generate_doc("Supplemental Appendix", "austcovid")
compile_doc(doc_sections, supplement)

In [ ]:
if on_colab:
    # To build a PDF, we need the appropriate tex packages installed
    ! apt-get -y install texlive-latex-base texlive-fonts-recommended texlive-fonts-extra texlive-latex-extra texlive-bibtex-extra biber
    # To avoid clutter and navigate to the right directory
    import os
    os.chdir("supplement")
    # And finally build the formatted PDF, repeated commands are necessary
    ! pdflatex supplement
    ! biber supplement
    ! pdflatex supplement
    ! pdflatex supplement